In [20]:
import numpy as np

In [21]:
def triangle(x, a, b, c):
    return max(min((x-a)/(b-a), (c-x)/(c-b)), 0)

In [22]:
def trapazoid(x, a, b, c, d):
    return max(min((x-a)/(b-a), 1, (d-x)/(d-c)), 0)

In [23]:
def fuzzify_temp(temp):
    cold = triangle(temp, -10, 0, 20)
    warm = triangle(temp, 15, 25, 35)
    hot = triangle(temp, 30, 40, 50)
    return {'cold':cold, 'warm':warm, 'hot': hot}

In [24]:
def fuzzify_err(err):
    negative = triangle(err, -10, -5, 0)
    zero = triangle(err, -2, 0, 2)
    positive = triangle(err, 0, 5, 10)
    return {'negative': negative, 'zero':zero, 'positive':positive}

In [25]:
def rule_eval(temp_mf, err_mf):
    rules = {
        ('cold', 'positive'): 'high',
        ('warm', 'zero'): 'medium',
        ('hot', 'negative'): 'low',
    }
    
    heater_mf = {'low': 0, 'medium': 0, 'high': 0}
    
    for (temp_key, err_key), heater_key in rules.items():
        activation = min(temp_mf[temp_key], err_mf[err_key])
        heater_mf[heater_key] = max(heater_mf[heater_key], activation)
        
    return heater_mf

In [26]:
def defuzzify_heater(heater_mf):
    def heater_low(x): return triangle(x, 70, 90, 100)
    def heater_medium(x): return triangle(x, 40, 60, 80)
    def heater_high(x): return triangle(x, 10, 30, 50)
    
    aggregated = lambda x:max(
        heater_low(x)*heater_mf['low'],
        heater_medium(x)*heater_mf['medium'],
        heater_high(x)*heater_mf['high']
    )
    
    resolution = 1000
    x_vals = np.linspace(0, 100, resolution)
    numerator = sum(x*aggregated(x) for x in x_vals)
    denominator = sum(aggregated(x) for x in x_vals)
    
    return numerator/denominator if denominator !=0 else 0

In [37]:
def flc(temp, err):
    temp_mf = fuzzify_temp(temp)
    err_mf = fuzzify_err(err)
    heater_mf = rule_eval(temp_mf, err_mf)
    heater_power = defuzzify_heater(heater_mf)
    return heater_power

In [38]:
# if __name__ == "main":
temp = 18
err = 3
heater_power = flc(temp, err)
print(f"Heater Power: {heater_power:.2f}%")

Heater Power: 30.00%
